# Mini-projeto 03 — Pandas para EDA

Limpeza + merges + perguntas de negócio em uma EDA organizada.

> Entregável de portfólio — gerado em 2026-01-30.


## Objetivo

        Construir uma EDA com tipagem, joins (sales + customers) e agregações úteis para negócio.

        ## Entregáveis (para portfólio)

        - [ ] `reports/eda_pandas.md`
- [ ] `assets/revenue_by_category.png`

        ## Como usar

        1- Rode este notebook (kernel com o `.venv` do repo)  
        2- Gere **assets** (imagens/HTML) e **reports** (markdown/json) dentro desta pasta  
        3- Faça commit dos arquivos gerados para evidenciar o resultado no GitHub

In [ ]:
# Setup: detectar raiz do repositório + paths padrão
from pathlib import Path
import pandas as pd
import numpy as np

def find_repo_root(start: Path | None = None) -> Path:
    p = (start or Path.cwd()).resolve()
    for _ in range(12):
        if (p / "requirements.txt").exists() and (p / "README.md").exists():
            return p
        p = p.parent
    return (start or Path.cwd()).resolve()

ROOT = find_repo_root()
DATA_SAMPLE = ROOT / "data" / "sample"
DATA_SOURCE = ROOT / "data" / "source" / "bases-dados-analytics-powerbi-ml"

# Pasta do projeto (onde salvar assets/outputs/reports)
PROJ = ROOT / "projects" / "03_pandas"
ASSETS = PROJ / "assets"
OUTPUTS = PROJ / "outputs"
REPORTS = PROJ / "reports"
for d in (ASSETS, OUTPUTS, REPORTS):
    d.mkdir(parents=True, exist_ok=True)

# Dataset padrão (sempre disponível)
sales_path = DATA_SAMPLE / "sales.csv"
customers_path = DATA_SAMPLE / "customers.csv"
sales = pd.read_csv(sales_path, parse_dates=["date"])
customers = pd.read_csv(customers_path, parse_dates=["signup_date"])

# Dataset real (opcional): se você adicionou o submodule/clone em dados/source/
has_real = DATA_SOURCE.exists()

print("ROOT:", ROOT)
print("Dataset sample:", sales.shape, customers.shape)
print("Dataset real disponível?", has_real)

In [ ]:
import matplotlib.pyplot as plt

# 1) Merge
df = sales.merge(customers, on="customer_id", how="left")

# 2) Checks
df["date"] = pd.to_datetime(df["date"], errors="coerce")
assert df["date"].notna().all(), "Datas inválidas após parse."

# 3) Agregações
by_cat = df.groupby("category", as_index=False)["revenue"].sum().sort_values("revenue", ascending=False)
by_seg = df.groupby("segment", as_index=False)["revenue"].sum().sort_values("revenue", ascending=False)

# 4) Plot
plt.figure()
plt.bar(by_cat["category"], by_cat["revenue"])
plt.title("Receita por categoria")
plt.xticks(rotation=30, ha="right")
out_img = ASSETS / "revenue_by_category.png"
plt.tight_layout()
plt.savefig(out_img, dpi=160)
plt.close()
print("Salvo:", out_img)

# 5) Report em Markdown
md = []
md.append("# EDA — 03_pandas\n")
md.append(f"- Linhas após merge: {len(df)}\n")
md.append("## Receita por categoria\n")
md.append(by_cat.to_markdown(index=False))
md.append("\n## Receita por segmento\n")
md.append(by_seg.to_markdown(index=False))
out = REPORTS / "eda_pandas.md"
out.write_text("\n".join(md), encoding="utf-8")
print("Salvo:", out)

## Evidências

![](./assets/revenue_by_category.png)
